In [1]:
from grtoolkit.Math import *
import re

In [2]:
expr = 'eq.append("Eq(w,integrate(C*v,(v,v0,v1))-integrate(C*v,(v,v0,v1))+integrate(8*diff(v**2,v),(v,v0,vt), (v4,f3)))")'
# print(preSympifySub(expr,v="99*red_ballons"))
expr.find("integrate")

16

In [16]:
def gen_sorted_bracket_pairs(expr): #ordered bracket pairs
    open_brackets = [m.start() for m in re.finditer('\(', expr)]
    close_brackets = [m.start() for m in re.finditer('\)', expr)]
    copyOpen = open_brackets
    taken = list()
    bracket_pairs = list()
    
    all_open_brackets = [[open,'o'] for open in open_brackets]
    all_close_brackets = [[close,'c'] for close in close_brackets]
    all_brackets = sorted(all_open_brackets + all_close_brackets)

    for b in range(len(all_brackets)):
        if all_brackets[b][1] == "c":
            i = 1
            while True:
                if all_brackets[b-i][1] == "o" and b-i not in taken:
                    bracket_pairs.append([all_brackets[b-i][0],all_brackets[b][0]])
                    taken.append(b-i)
                    break
                i+=1
    sorted_bracket_pairs = sorted(bracket_pairs)
    # print(sorted(sorted_bracket_pairs)) ### BRACKET PAIRS
    return sorted_bracket_pairs


In [4]:
# test = "(tomato{ went } down the ) (street)"
#note: bug in working with incomplete pairs of brackets
# print(obp(test))

# sorted_bracket_pairs = gen_sorted_bracket_pairs(expr)
# print(sorted_bracket_pairs)

[[9, 112], [13, 110], [25, 39], [30, 38], [50, 64], [55, 63], [75, 109], [82, 89], [91, 99], [102, 108]]


In [17]:
def gen_func_bps(func_name, expr, sorter_bracket_pairs): #bracket pair after function namefunction opening and closing bracket pairs in str
    funcs = [m.start() for m in re.finditer(func_name, expr)]
    master_func_bracket_pairs = list()

    for i in range(len(sorted_bracket_pairs)):
    # print(bracket_pairs[i])
        for func in funcs:
            if func > sorted_bracket_pairs[i][0] and func < sorted_bracket_pairs[i+1][0]:
                master_func_bracket_pairs.append(sorted_bracket_pairs[i+1])
    return master_func_bracket_pairs

In [18]:
func_bps = gen_func_bps("integrate", expr, sorted_bracket_pairs)
print(func_bps)

[[25, 39], [50, 64], [75, 109]]


In [8]:
# #find bracket pairs inside func_bps:
# func_sub_bps = list()
# for fbp in func_bps:
#     temp_sub_bps = list()
#     for bp in sorted_bracket_pairs:
#         if bp[0]>fbp[0] and bp[1]<fbp[1]:
#             temp_sub_bps.append(bp)
#     func_sub_bps.append(temp_sub_bps)


In [14]:
# #visualizing sub bracket pairs
# func_sub_bps
# print(expr)
# for a in func_sub_bps:
#     for b in a:
#         print(expr[b[0]:b[1]+1])

In [10]:
# #find bracket pairs inside func_bps:
# func_sub_bps = list()
# for fbp in func_bps:
#     temp_sub_bps = list()
#     for bp in sorted_bracket_pairs:
#         if bp[0]>fbp[0] and bp[1]<fbp[1]:
#             temp_sub_bps.append(bp)
#     func_sub_bps.append(temp_sub_bps)

# #find commas inside function bracket pairs that are not inside function sub bracket pairs
# commas = [m.start() for m in re.finditer(',', expr)]
# func_comma = list()
# ## for each comma:
# i=0
# for fbp in func_bps:
#     temp_comma = list()
#     for comma in commas:
#     ## if comma within function bracket pair:
#         if comma > fbp[0] and comma < fbp[1]: #if comma within function brackets
#             AND_counter = list()
#             for sfbp in func_sub_bps[i]: # if comma not within any bracket pair within function
#                 if comma < sfbp[0] or comma > sfbp[1]:  # is comma outside of this sub-bracket?
#                     AND_counter.append(1)
#                 else:
#                     AND_counter.append(0)
#             if sum(AND_counter) == len(AND_counter): # append comma only when outside all sub-brackets
#                 temp_comma.append(comma)
#     if comma:
#         func_comma.append(temp_comma)
#     i+=1

# #generating list of ranges to protect
# zipped_list = list(zip(func_bps,func_comma))
# ranges2protect = list()
# i=0
# for func, fcommas in zipped_list:
#     i=0
#     mini_r2p=list()
#     for fcomma in fcommas:
#         if (len(fcommas)-i)>1:
#             # print(expr[fcommas[i]+1:fcommas[i+1]])
#             mini_r2p.append([fcommas[i]+1, fcommas[i+1]])
#         else:
#             # print(expr[fcomma+1:func[1]])
#             mini_r2p.append([fcomma+1,func[1]])
#         i+=1
#     ranges2protect.append(mini_r2p)

In [19]:
def gen_func_sub_bps(func_bps,sorted_bracket_pairs): 
    """function sub bracket pairs"""
    #Find bracket pairs inside func_bps:
    func_sub_bps = list()
    for fbp in func_bps:
        temp_sub_bps = list()
        for bp in sorted_bracket_pairs:
            if bp[0]>fbp[0] and bp[1]<fbp[1]:
                temp_sub_bps.append(bp)
        func_sub_bps.append(temp_sub_bps)
    return func_sub_bps

def gen_func_commas(expr, func_bps, func_sub_bps):
    #Find commas inside function bracket pairs that are not inside function sub bracket pairs
    commas = [m.start() for m in re.finditer(',', expr)]
    func_commas = list()
    ## for each comma:
    i=0
    for fbp in func_bps:
        temp_comma = list()
        for comma in commas:
        ## if comma within function bracket pair:
            if comma > fbp[0] and comma < fbp[1]: #if comma within function brackets
                AND_counter = list()
                for sfbp in func_sub_bps[i]: # if comma not within any bracket pair within function
                    if comma < sfbp[0] or comma > sfbp[1]:  # is comma outside of this sub-bracket?
                        AND_counter.append(1)
                    else:
                        AND_counter.append(0)
                if sum(AND_counter) == len(AND_counter): # append comma only when outside all sub-brackets
                    temp_comma.append(comma)
        if comma:
            func_commas.append(temp_comma)
        i+=1
    return func_commas

def gen_ranges2protect(func_bps, func_comma):
    #generating list of ranges to protect
    zipped_list = list(zip(func_bps,func_comma))
    ranges2protect = list()
    i=0
    for func, fcommas in zipped_list:
        i=0
        mini_r2p=list()
        for fcomma in fcommas:
            if (len(fcommas)-i)>1:
                # print(expr[fcommas[i]+1:fcommas[i+1]])
                mini_r2p.append([fcommas[i]+1, fcommas[i+1]])
            else:
                # print(expr[fcomma+1:func[1]])
                mini_r2p.append([fcomma+1,func[1]])
            i+=1
        ranges2protect.append(mini_r2p)
    return ranges2protect

In [12]:
ranges2protect

[[[30, 39]], [[55, 64]], [[91, 100], [101, 109]]]

In [25]:
expr = 'eq.append("Eq(w,integrate(C*v,(v,v0,v1))-integrate(C*v,(v,v0,v1))+integrate(8*diff(v**2,v),(v,v0,vt), (v4,f3)))")'
# print(preSympifySub(expr,v="99*red_ballons"))
# expr.find("integrate")

sorted_bracket_pairs = gen_sorted_bracket_pairs(expr)
func_bps = gen_func_bps("integrate", expr, sorted_bracket_pairs)
sub_bps = gen_func_sub_bps(func_bps,sorted_bracket_pairs)
func_commas = gen_func_commas(expr, func_bps, func_sub_bps)
gen_ranges2protect(func_bps, func_commas)

[[[30, 39]], [[55, 64]], [[91, 100], [101, 109]]]